classical_models.ipynb

by: Archie Gertsman (arkadiy2@illinois.edu) Lloyd Fernandes (lloydf2@illinois.edu)

Project director: Richard Sowers

r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/

Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license

In [ ]:
####Simple_workflow

In [1]:

import sys
sys.path.append('../src/')
sys.path.append('../data/')

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from feature_eng import split_trajectories
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from model_functions import *
import seaborn as sns

In [2]:
df = pd.read_pickle('block4_updated.pkl') 
 
df.head()

lat        lon    speed  \
file_name id edge_id             time                                   
4_1       1  250699362_250699984 42.00  37.982746  23.732961  11.9046   
                                 42.04  37.982746  23.732963  11.8975   
                                 42.08  37.982747  23.732964  11.8919   
                                 42.12  37.982748  23.732965  11.8871   
                                 42.16  37.982748  23.732966  11.8831   

                                        lon_acc  lat_acc  type  traveled_d  \
file_name id edge_id             time                                        
4_1       1  250699362_250699984 42.00  -0.1145   0.0138  Taxi      182.37   
                                 42.04  -0.1007   0.0147  Taxi      182.37   
                                 42.08  -0.0918   0.0157  Taxi      182.37   
                                 42.12  -0.0869   0.0167  Taxi      182.37   
                                 42.16  -0.0784   0.0176  Taxi      182.37   

                                        avg_speed   bearing  \
file_name id edge_id             time                         
4_1       1  250699362_250699984 42.00   9.740748  1.570795   
                                 42.04   9.740748  0.168572   
                                 42.08   9.740748  0.168573   
                                 42.12   9.740748  1.570796   
                                 42.16   9.740748  0.328080   

                                        nearest_edge_start_node  ...  \
file_name id edge_id             time                            ...   
4_1       1  250699362_250699984 42.00                250699362  ...   
                                 42.04                250699362  ...   
                                 42.08                250699362  ...   
                                 42.12                250699362  ...   
                                 42.16                250699362  ...   

                                        vehicle_density  avg_surr_speed  \
file_name id edge_id             time                                     
4_1       1  250699362_250699984 42.00                7       10.464171   
                                 42.04                7       10.457843   
                                 42.08                7       10.452857   
                                 42.12                7       10.448586   
                                 42.16                7       10.444986   

                                        edge_bearing  acc_edge  acc_per_edge  \
file_name id edge_id             time                                          
4_1       1  250699362_250699984 42.00      -2.83013  0.113220      0.021953   
                                 42.04      -2.83013  0.100360      0.016867   
                                 42.08      -2.83013  0.092194      0.013188   
                                 42.12      -2.83013  0.087837      0.010734   
                                 42.16      -2.83013  0.080021      0.007273   

                                        xtrack_diff  xtrack_diff_sq  \
file_name id edge_id             time                                 
4_1       1  250699362_250699984 42.00          0.0             0.0   
                                 42.04          0.0             0.0   
                                 42.08          0.0             0.0   
                                 42.12          0.0             0.0   
                                 42.16          0.0             0.0   

                                        acc_edge_sq  acc_per_edge_sq  \
file_name id edge_id             time                                  
4_1       1  250699362_250699984 42.00     0.012819         0.000482   
                                 42.04     0.010072         0.000284   
                                 42.08     0.008500         0.000174   
                                 42.12     0.007715         0.000115   
                       

In [3]:
#initial parameters

agg_dict = {
            'xtrack_diff': ['mean','std'],
            'xtrack_dist': ['mean','std'],
            'avg_surr_speed': ['mean','std'],
            'lanes':['mean'],
            'len':['mean'],
            'speed':['mean','std'],
            'speed_bool': ['count','sum'],
            'lat_acc': ['mean','std'],
            'lon_acc': ['mean','std']
            }

overlap = 0.3
min_movement_limit = 1
speed_limit = 0
k = 5

traj_lens = np.arange(50,250 ,step=50)

validation_ratio = 0.2
vehicle_density = 1


In [ ]:
for i in range(0,10):
    traj_len = 50
    model = Pipeline([('scaler', StandardScaler()), ('gbm', GradientBoostingClassifier())])
    """ 
    Fit model to classify among cars and taxis. 
    1. Remove vehicles with length less than traj_len
    2. split trajectory to training and testing set
    3. get training and testing aggregates 
    4. fit the model
    """
    df_filtered = df.groupby(df.index.names[:-1]) \
            .filter(lambda grp: (len(grp) >= traj_len) )

    df_filtered['speed_bool'] = (df_filtered['speed']>speed_limit).astype(int)
    
    df_train,df_test = split_train_test(df_filtered,validation_ratio)

    #aggregate trajectories
    #to train models
    X_train,y_train = get_xy(df_train,overlap = overlap,traj_len = traj_len,agg_dict = agg_dict,outlier_limit = 1,balance = 'by_edge')
    #to pick better performing models
    X_test,y_test = get_xy(df_test,overlap = overlap,traj_len = traj_len,agg_dict = agg_dict)

    model.fit(X_train,y_train)
    
    accuracy,_ = voting_accuracy(X_test,y_test, model,predict_proba = True)
    print("voting accuracy Car vs Taxi: ",accuracy)
    

    """ 
    Fit model to classify among cars. 
    1. Select 50% cars and label them as Car_1
    2. follow steps 1-4 as above
    """
    
    df_car = df[df.type == 'Car'].copy()
    df_car_filtered = df_car.groupby(df_car.index.names[:-1]) \
            .filter(lambda grp: (len(grp) >= traj_len) )
    df_car_filtered['speed_bool'] = (df_car_filtered['speed']>speed_limit).astype(int)

    df_index = df_car_filtered.reset_index()[['file_name','id']].drop_duplicates()
    df_car_filtered.loc[df_car_filtered.reset_index(['edge_id', 'time'],drop = True).index.isin(df_index.sample(frac = 0.5).set_index(['file_name','id']).index),'type']='Car_1'

    df_train,df_test = split_train_test(df_car_filtered,validation_ratio)

    #to train models
    X_train,y_train = get_xy(df_train,overlap = overlap,traj_len = traj_len,agg_dict = agg_dict,outlier_limit = 1,balance = 'by_edge')
    #to pick better performing models
    X_test,y_test = get_xy(df_test,overlap = overlap,traj_len = traj_len,agg_dict = agg_dict)


    model.fit(X_train,y_train)
    accuracy,_ = voting_accuracy(X_test,y_test, model,predict_proba = True)
    print("voting accuracy Car vs Car: ",accuracy)
    print("\n")


voting accuracy Car vs Taxi:  0.6190476190476191
voting accuracy Car vs Car:  0.5043731778425656


voting accuracy Car vs Taxi:  0.6351351351351351
voting accuracy Car vs Car:  0.5391304347826087


voting accuracy Car vs Taxi:  0.6245733788395904
voting accuracy Car vs Car:  0.4941860465116279


voting accuracy Car vs Taxi:  0.6508474576271186
voting accuracy Car vs Car:  0.5102040816326531


voting accuracy Car vs Taxi:  0.6655290102389079
voting accuracy Car vs Car:  0.4508670520231214


voting accuracy Car vs Taxi:  0.6508474576271186
voting accuracy Car vs Car:  0.5375722543352601


voting accuracy Car vs Taxi:  0.6292517006802721


In [22]:
df_acc.sort_index().to_csv('agg_wo_pca.csv')
df_acc.sort_index().to_pickle('agg_wo_pca.pkl')